In [1]:
import os 
import unicodedata
import json
from django.contrib.gis.geos import GEOSGeometry
import time 
import shutil 

from shapely.geometry import mapping
import pandas as pd
import fiona
import geopandas as gpd
from fiona.crs import to_string
from fiona.crs import from_epsg
from shapely.geometry import shape, Point, Polygon
from fiona.crs import to_string

from madmex.overlay.conversions import train_object_to_feature
from madmex.models import Tag
from madmex.util.spatial import geometry_transform
from madmex.models import TrainClassification

Checar primero:

https://github.com/CONABIO/antares3-sandbox/blob/master/notebooks/agriculture_madmex_app/3a_assign_labels_and_insert_in_db_for_agriculture.ipynb

# 7) Random sample para cada categoría generada de Kmeans

**La muestra debe generarse de modo que incluya los polígonos dados por inecol más otros y que en total sean 1000**

Intersección y contención del geopandas dataframe con el shapefile:

In [2]:
input_file2='/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4.shp'

In [3]:
gdf_df_clusters = gpd.read_file(input_file2)

In [4]:
gdf_df_clusters.head()

,features_i,preds,geometry
0,537885,2,"POLYGON ((2977960.000 926120.000, 2977980.000 ..."
1,537675,2,"POLYGON ((2978250.000 926120.000, 2978420.000 ..."
2,537799,0,"POLYGON ((2978440.000 926120.000, 2978510.000 ..."
3,537676,2,"POLYGON ((2978610.000 926120.000, 2978650.000 ..."
4,537886,2,"POLYGON ((2979320.000 926120.000, 2979340.000 ..."


In [5]:
len(gdf_df_clusters.index)

208008

In [6]:
gdf_df_clusters.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True}

**Leer puntos de cultivos ya registrados con su valor numérico en la tabla TrainClassification**

Obtener extent en coordenadas geográficas de polígonos clúster

https://gis.stackexchange.com/questions/266730/filter-by-bounding-box-in-geopandas

http://geopandas.org/set_operations.html

In [7]:
bbox = gdf_df_clusters.total_bounds

In [8]:
p1 = Point(bbox[0], bbox[3])
p2 = Point(bbox[2], bbox[3])
p3 = Point(bbox[2], bbox[1])
p4 = Point(bbox[0], bbox[1])
p1 = shape(geometry_transform(mapping(p1),crs_out="+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",
                              crs_in=gdf_df_clusters.crs))
p2 = shape(geometry_transform(mapping(p2),crs_out="+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",
                              crs_in=gdf_df_clusters.crs))
p3 = shape(geometry_transform(mapping(p3),crs_out="+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",
                              crs_in=gdf_df_clusters.crs))
p4 = shape(geometry_transform(mapping(p4),crs_out="+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",
                              crs_in=gdf_df_clusters.crs))
np1 = (p1.coords.xy[0][0], p1.coords.xy[1][0])
np2 = (p2.coords.xy[0][0], p2.coords.xy[1][0])
np3 = (p3.coords.xy[0][0], p3.coords.xy[1][0])
np4 = (p4.coords.xy[0][0], p4.coords.xy[1][0])

In [9]:
bb_polygon = Polygon([np1, np2, np3, np4])

In [10]:
bb_polygon_for_query = GEOSGeometry(json.dumps(mapping(bb_polygon)))

In [11]:
bb_polygon_for_query

In [12]:
training_set = 'train_cultivos_inecol'

In [13]:
query_set = TrainClassification.objects.filter(train_object__the_geom__contained=bb_polygon_for_query,
                                               training_set=training_set).prefetch_related('train_object', 'interpret_tag')

In [14]:
len(query_set)

676

In [15]:
fc_cultivos = [train_object_to_feature(x, to_string(gdf_df_clusters.crs)) for x in query_set]

In [16]:
fc_cultivos[0]

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [3007166.7513802242, 884499.104599742]},
 'properties': {'class': 54}}

In [17]:
# Build the GeoDataFrame from Fiona Collection
gdf_poi = gpd.GeoDataFrame.from_features([feature for feature in fc_cultivos], crs=gdf_df_clusters.crs)


In [18]:
gdf_poi.head()

,geometry,class
0,POINT (3007166.751 884499.105),54
1,POINT (3004236.090 883061.625),57
2,POINT (2999113.725 878073.927),58
3,POINT (3007515.715 884292.293),60
4,POINT (3011360.808 883057.195),60


In [19]:
gdf_poi.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True}

**Interesan los id's que intersectan o contienen los puntos pues se extraerán tales id's del gdf_df_clusters y a los restantes se les extraerá una muestra**

Obtener `features_id` de `gdf_df_clusters`  que intersectan o contienen `gdf_poi`

In [20]:
gdf_df_clusters_contains_poi = gpd.sjoin(gdf_df_clusters,gdf_poi,op='contains').drop_duplicates(subset='features_i')

In [21]:
gdf_df_clusters_contains_poi.head()

,features_i,preds,geometry,index_right,class
1280,534719,0,"POLYGON ((2984820.000 925930.000, 2984880.000 ...",274,55
1301,534675,3,"POLYGON ((3014490.000 926050.000, 3014490.000 ...",530,54
1865,533540,3,"POLYGON ((2982510.000 926030.000, 2982540.000 ...",302,55
2142,533080,0,"POLYGON ((3015370.000 925730.000, 3015390.000 ...",527,54
2900,531041,0,"POLYGON ((3016300.000 925690.000, 3016350.000 ...",164,59


In [22]:
len(gdf_df_clusters_contains_poi['features_i'])

647

In [23]:
gdf_df_clusters_poi_intersects = gpd.sjoin(gdf_df_clusters,gdf_poi,op='intersects').drop_duplicates(subset='features_i')

In [24]:
gdf_df_clusters_poi_intersects.head()

,features_i,preds,geometry,index_right,class
1280,534719,0,"POLYGON ((2984820.000 925930.000, 2984880.000 ...",274,55
1301,534675,3,"POLYGON ((3014490.000 926050.000, 3014490.000 ...",530,54
1865,533540,3,"POLYGON ((2982510.000 926030.000, 2982540.000 ...",302,55
2142,533080,0,"POLYGON ((3015370.000 925730.000, 3015390.000 ...",527,54
2900,531041,0,"POLYGON ((3016300.000 925690.000, 3016350.000 ...",164,59


In [25]:
gdf_df_clusters_poi_intersects.head()

,features_i,preds,geometry,index_right,class
1280,534719,0,"POLYGON ((2984820.000 925930.000, 2984880.000 ...",274,55
1301,534675,3,"POLYGON ((3014490.000 926050.000, 3014490.000 ...",530,54
1865,533540,3,"POLYGON ((2982510.000 926030.000, 2982540.000 ...",302,55
2142,533080,0,"POLYGON ((3015370.000 925730.000, 3015390.000 ...",527,54
2900,531041,0,"POLYGON ((3016300.000 925690.000, 3016350.000 ...",164,59


In [26]:
len(gdf_df_clusters_poi_intersects.index)

647

In [27]:
t_intersects_rows = tuple(gdf_df_clusters_poi_intersects['features_i'])

In [28]:
gdf_df_clusters_contains_poi_reduced = gpd.GeoDataFrame()

In [29]:
gdf_df_clusters_contains_poi_reduced['features_i'] = gdf_df_clusters_contains_poi['features_i']
gdf_df_clusters_contains_poi_reduced['preds'] = gdf_df_clusters_contains_poi['preds']
gdf_df_clusters_contains_poi_reduced['geometry'] = gdf_df_clusters_contains_poi['geometry']
gdf_df_clusters_contains_poi_reduced['class'] = gdf_df_clusters_contains_poi['class']

In [30]:
gdf_df_clusters_contains_poi_reduced.head()

,features_i,preds,geometry,class
1280,534719,0,"POLYGON ((2984820.000 925930.000, 2984880.000 ...",55
1301,534675,3,"POLYGON ((3014490.000 926050.000, 3014490.000 ...",54
1865,533540,3,"POLYGON ((2982510.000 926030.000, 2982540.000 ...",55
2142,533080,0,"POLYGON ((3015370.000 925730.000, 3015390.000 ...",54
2900,531041,0,"POLYGON ((3016300.000 925690.000, 3016350.000 ...",59


In [31]:
len(gdf_df_clusters_contains_poi_reduced.index)

647

In [32]:
sum(gdf_df_clusters_contains_poi_reduced['features_i'].isin(t_intersects_rows))

647

In [33]:
gdf_df_clusters_keep = gdf_df_clusters_contains_poi_reduced.loc[gdf_df_clusters_contains_poi_reduced['features_i'].isin(t_intersects_rows)]

In [34]:
gdf_df_clusters_keep.head()

,features_i,preds,geometry,class
1280,534719,0,"POLYGON ((2984820.000 925930.000, 2984880.000 ...",55
1301,534675,3,"POLYGON ((3014490.000 926050.000, 3014490.000 ...",54
1865,533540,3,"POLYGON ((2982510.000 926030.000, 2982540.000 ...",55
2142,533080,0,"POLYGON ((3015370.000 925730.000, 3015390.000 ...",54
2900,531041,0,"POLYGON ((3016300.000 925690.000, 3016350.000 ...",59


In [35]:
len(gdf_df_clusters_keep.index)

647

**If number of rows are less than 1000 then filter_out rows that don't contain nor intersect points and then sample**

In [36]:
t_keep_rows = tuple(gdf_df_clusters_keep['features_i'])

In [37]:
len(t_keep_rows)

647

In [38]:
t_keep_rows[0:5]

(534719, 534675, 533540, 533080, 531041)

In [39]:
gdf_df_clusters_to_sample = gdf_df_clusters.loc[~gdf_df_clusters['features_i'].isin(t_keep_rows)]

In [40]:
gdf_df_clusters_to_sample.head()

,features_i,preds,geometry
0,537885,2,"POLYGON ((2977960.000 926120.000, 2977980.000 ..."
1,537675,2,"POLYGON ((2978250.000 926120.000, 2978420.000 ..."
2,537799,0,"POLYGON ((2978440.000 926120.000, 2978510.000 ..."
3,537676,2,"POLYGON ((2978610.000 926120.000, 2978650.000 ..."
4,537886,2,"POLYGON ((2979320.000 926120.000, 2979340.000 ..."


In [41]:
len(gdf_df_clusters_to_sample.index)

207361

**Extraer muestra**

Using https://stackoverflow.com/questions/41345289/getting-a-random-sample-in-python-dataframe-by-category

In [42]:
gdf_df_clusters_to_sample.head()

,features_i,preds,geometry
0,537885,2,"POLYGON ((2977960.000 926120.000, 2977980.000 ..."
1,537675,2,"POLYGON ((2978250.000 926120.000, 2978420.000 ..."
2,537799,0,"POLYGON ((2978440.000 926120.000, 2978510.000 ..."
3,537676,2,"POLYGON ((2978610.000 926120.000, 2978650.000 ..."
4,537886,2,"POLYGON ((2979320.000 926120.000, 2979340.000 ..."


In [43]:
nmultipolygons=len(gdf_df_clusters_to_sample.index)

In [44]:
nmultipolygons

207361

In [45]:
total_mpolygons = 1000
nsample=total_mpolygons-len(t_keep_rows)

In [46]:
nsample

353

In [47]:
nclusters=4

In [48]:
nclusters

4

In [49]:
if nmultipolygons>nsample:
    nsample_per_cluster=int(nsample/nclusters)
    nsample=nsample_per_cluster*nclusters
else:
    nsample_per_cluster=int(nmultipolygons/nclusters)
    nsample=nsample_per_cluster*nclusters

In [50]:
(nmultipolygons,nsample_per_cluster,nsample_per_cluster*nclusters,nsample)

(207361, 88, 352, 352)

In [51]:
gdf_df_clusters_sample=gdf_df_clusters_to_sample.groupby('preds').apply(lambda s: s.sample(min(len(s),
                                                                                               nsample_per_cluster),
                                                                                          random_state=1))

In [52]:
gdf_df_clusters_sample.head()

features_i  preds  \
preds                             
0     60570       375200      0   
      26750       461215      0   
      196538       39041      0   
      108579      278994      0   
      93347       307663      0   

                                                       geometry  
preds                                                            
0     60570   POLYGON ((3022310.000 912080.000, 3022330.000 ...  
      26750   POLYGON ((3017340.000 919570.000, 3017360.000 ...  
      196538  POLYGON ((3000000.000 879680.000, 3000050.000 ...  
      108579  POLYGON ((3001140.000 903840.000, 3001150.000 ...  
      93347   POLYGON ((3006580.000 906450.000, 3006640.000 ...

In [53]:
gdf_df_clusters_sample.reset_index(drop=True,inplace=True)

In [54]:
gdf_df_clusters_sample.head()

,features_i,preds,geometry
0,375200,0,"POLYGON ((3022310.000 912080.000, 3022330.000 ..."
1,461215,0,"POLYGON ((3017340.000 919570.000, 3017360.000 ..."
2,39041,0,"POLYGON ((3000000.000 879680.000, 3000050.000 ..."
3,278994,0,"POLYGON ((3001140.000 903840.000, 3001150.000 ..."
4,307663,0,"POLYGON ((3006580.000 906450.000, 3006640.000 ..."


In [55]:
len(gdf_df_clusters_sample.index)

352

**Union of sample and keep polygons**

In [56]:
gdf_sampled_keep = pd.concat([gdf_df_clusters_sample,gdf_df_clusters_keep],
                             ignore_index=True,sort=False).pipe(gpd.GeoDataFrame)

In [57]:
gdf_sampled_keep.head()

,features_i,preds,geometry,class
0,375200,0,"POLYGON ((3022310.000 912080.000, 3022330.000 ...",NaN
1,461215,0,"POLYGON ((3017340.000 919570.000, 3017360.000 ...",NaN
2,39041,0,"POLYGON ((3000000.000 879680.000, 3000050.000 ...",NaN
3,278994,0,"POLYGON ((3001140.000 903840.000, 3001150.000 ...",NaN
4,307663,0,"POLYGON ((3006580.000 906450.000, 3006640.000 ...",NaN


In [58]:
sum(gdf_sampled_keep['class'].isna())

352

In [59]:
gdf_sampled_keep.loc[gdf_sampled_keep['class'].isna(),'class'] = ''

In [60]:
sum(gdf_sampled_keep['class'].isna())

0

In [61]:
gdf_sampled_keep.head()

,features_i,preds,geometry,class
0,375200,0,"POLYGON ((3022310.000 912080.000, 3022330.000 ...",
1,461215,0,"POLYGON ((3017340.000 919570.000, 3017360.000 ...",
2,39041,0,"POLYGON ((3000000.000 879680.000, 3000050.000 ...",
3,278994,0,"POLYGON ((3001140.000 903840.000, 3001150.000 ...",
4,307663,0,"POLYGON ((3006580.000 906450.000, 3006640.000 ...",


In [62]:
len(gdf_sampled_keep.index)

999

# 8) Write results to file

In [63]:
date_today = '02-01-2020'
dc_tile = '40_-31'
path_result_dc_tile = '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/'
features_string = 'ndvi_mean-ndvi_min-ndvi_max'

**Geopackage**

In [64]:
path_results_geopackage = '/home/madmex_user/agriculture_geopackages/features_computed/' + features_string + '/'

In [65]:
path_results_geopackage

'/home/madmex_user/agriculture_geopackages/features_computed/ndvi_mean-ndvi_min-ndvi_max/'

In [66]:
path_result_geopackage = os.path.join(path_results_geopackage, date_today 
                                       + '/' + dc_tile + '/')

In [67]:
path_result_geopackage

'/home/madmex_user/agriculture_geopackages/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/'

In [68]:
if not os.path.exists(path_result_geopackage):
    os.makedirs(path_result_geopackage)

In [69]:
clusters_filename = path_result_geopackage + date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_sample_%d' % total_mpolygons + '_with_class.gpkg'

In [70]:
clusters_filename

'/home/madmex_user/agriculture_geopackages/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4_sample_1000_with_class.gpkg'

In [71]:
layer_name = date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_sample_%d' % total_mpolygons + '_with_class'

In [72]:
layer_name

'02-01-2020_clusters_40_-31_nclusters_4_sample_1000_with_class'

In [73]:
gdf_sampled_keep.crs

In [74]:
gdf_sampled_keep.crs = gdf_poi.crs

In [75]:
gdf_sampled_keep.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True}

In [69]:
start_time=time.time()

gdf_sampled_keep.to_file(clusters_filename,
                         layer=layer_name,
                         driver='GPKG')
end_time=time.time()

In [70]:
secs = end_time-start_time
secs

0.35666966438293457

In [71]:
shutil.move(clusters_filename,path_result_dc_tile)

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4_sample_1000_with_class_2b.gpkg'

**Shapefile**

In [76]:
clusters_filename = path_result_dc_tile + date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_sample_%d' % total_mpolygons + '_with_class.shp'

In [77]:
clusters_filename

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4_sample_1000_with_class.shp'

In [78]:
gdf_sampled_keep.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True}

In [79]:
gdf_sampled_keep.crs = gdf_poi.crs

In [80]:
gdf_sampled_keep.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True}

In [77]:
start_time=time.time()
gdf_sampled_keep.to_file(clusters_filename)
end_time=time.time()

In [78]:
secs = end_time-start_time
secs

0.524092435836792

# 12) Registrar segmentos con etiquetas automáticas en trainclassificationlabeledbyapp con esquema "automatic_4_classes"

**Old command:**

In [81]:
%%bash
/home/madmex_user/.local/bin/antares ingest_training_from_vector \
/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4_sample_1000_with_class.shp \
--scheme automatic_4_classes --year 2020 --name agricultura_4_clases_sample_1000_with_class --field preds --app

Command line arguments: {'verbosity': 1, 'settings': None, 'pythonpath': None, 'traceback': False, 'no_color': False, 'force_color': False, 'input_file': '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4_sample_1000_with_class.shp', 'scheme': 'automatic_4_classes', 'field': 'preds', 'name': 'agricultura_4_clases_sample_1000_with_class', 'year': '2020', 'app': True}
Command execution is done in 13.483619928359985 seconds.
2020-02-06 20:55:45,824 - madmex.management.base - base execute: Command execution is done in 13.483619928359985 seconds.


**New command:**

In [89]:
%%bash
/home/madmex_user/.local/bin/antares ingest_training_from_vector \
/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4_sample_1000_with_class.shp \
--scheme automatic_4_classes --year 2020 --name agricultura_4_clases_sample_1000_with_class_added \
--field preds --app --train_interpreted --scheme_interpreted agricultura_completo --field_interpreted class

Command line arguments: {'verbosity': 1, 'settings': None, 'pythonpath': None, 'traceback': False, 'no_color': False, 'force_color': False, 'input_file': '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4_sample_1000_with_class.shp', 'scheme': 'automatic_4_classes', 'field': 'preds', 'name': 'agricultura_4_clases_sample_1000_with_class_added', 'year': '2020', 'app': True, 'train_interpreted': True, 'scheme_interpreted': 'agricultura_completo', 'field_interpreted': 'class'}
Command execution is done in 25.50695776939392 seconds.
2020-02-07 00:22:26,671 - madmex.management.base - base execute: Command execution is done in 25.50695776939392 seconds.
